In [1]:
import pandas as pd
import numpy as np
from scipy.stats import f
import statsmodels.api as sm

### Empirical Finance Tool
#### Data Upload

In [2]:
# Query the user for the file name, assuming file is already on the system
file_path = input("Enter the name of the data file including the .csv extension (already uploaded): ")

# Load the file and skipping rows without necessary information
df = pd.read_csv(file_path, skiprows=1)

# Rename the first column as 'Date'
df.rename(columns={df.columns[0]: 'Date'}, inplace=True)

# Drop unnamed rows
df = df.drop(columns=df.filter(regex="Unnamed").columns)

# Query user for number of time frames
nper = int(input('How many periods would you like to study? Please enter as an integer.'))

# Query the user for a date range in YYYYMM format
start_dates = []
end_dates = []
periods = []
for i in range(nper):
    start_date = input(f"Enter the start date (YYYYMM) for period {i+1}: ")
    start_dates.append(int(start_date))
    start_year = start_date[2:4]
    start_month = start_date[4:]
    formatted_start_date = start_month + '/' + start_year
    end_date = input(f"Enter the end date (YYYYMM) for period {i+1}: ")
    end_dates.append(int(end_date))
    end_year = end_date[2:4]
    end_month = end_date[4:]
    formatted_end_date = end_month + '/' + end_year
    formatted_period = formatted_start_date + ' - ' + formatted_end_date
    periods.append(formatted_period)

Enter the name of the data file including the .csv extension (already uploaded):  data.csv
How many periods would you like to study? Please enter as an integer. 2
Enter the start date (YYYYMM) for period 1:  196307
Enter the end date (YYYYMM) for period 1:  199306
Enter the start date (YYYYMM) for period 2:  199409
Enter the end date (YYYYMM) for period 2:  202408


## CAPM TEST

In [3]:
# Create a class to be used for all CAPM Calulcations
class EmpiricalTestsCAPM:
    def __init__(self, df):
        # Drop the 'Date' column
        self.df = df.drop(columns=['Date'], errors='ignore')

        # Store portfolio columns, ignoring the factors and RF column
        self.portfolio_columns = self.df.columns[:-4]
        self.RMRF = self.df['RMRF']
        self.SMB = self.df['SMB']
        self.HML = self.df['HML']
        self.RF = self.df['RF']
        self.excess_returns = pd.DataFrame()
        self.expected_returns = []


    # Preliminary functions for getting returns (excess and expected)
    def calculate_excess_returns(self):
        # Calculate the excess returns
        for column in self.portfolio_columns:
            self.excess_returns[column] = self.df[column] - self.RF
        print("Excess returns calculated.")

    def calculate_expected_returns(self):
        # Calculate the expected returns
        self.expected_returns = self.excess_returns.mean().values
        print("Expected returns calculated.")
        return self.expected_returns

    # Part 2c: Performing Time Series Regression
    def calculate_TSR_CAPM(self):
        # Ensure excess returns are calculated
        if self.excess_returns.empty:
            self.calculate_excess_returns()

        # Create dictionaries to store metrics
        self.capm_betas = {}
        self.capm_betas_se = {}
        self.capm_betas_tstat = {}
        self.capm_alphas = {}
        self.capm_alphas_se = {}
        self.capm_alphas_tstat = {}

        for column in self.portfolio_columns:
            # Perform a time-series regression
            X = sm.add_constant(self.RMRF)
            y = self.excess_returns[column]
            model = sm.OLS(y, X, missing='drop').fit()

            # Extract relevant metrics from the regression model
            self.capm_betas[column] = model.params['RMRF']
            self.capm_betas_se[column] = model.bse['RMRF']
            self.capm_betas_tstat[column] = model.tvalues['RMRF']
            self.capm_alphas[column] = model.params['const']
            self.capm_alphas_se[column] = model.bse['const']
            self.capm_alphas_tstat[column] = model.tvalues['const']

        # Print results for confirmation
        print("CAPM TSR metrics calculated.")
        return {
            "CAPM Betas": self.capm_betas,
            "Beta_SE": self.capm_betas_se,
            "Beta_tStat": self.capm_betas_tstat,
            "Alphas": self.capm_alphas,
            "Alpha_SE":self.capm_alphas_se,
            "Alpha_tStat": self.capm_alphas_tstat
        }

    # Part 2d: Perfoming Cross Sectional Regression
    def calculate_CSR_CAPM(self):
        # Conversions to ensure correct format
        # Using the CAPM betas
        capm_betas_series = pd.Series(self.capm_betas)
        expected_returns_series = pd.Series(self.expected_returns, index=capm_betas_series.index)

        # Perform a cross-sectional regression
        X = sm.add_constant(capm_betas_series)
        y = expected_returns_series
        model = sm.OLS(y, X).fit()

        # Extract relevant metrics
        lambda_value = model.params[0]
        lambda_se = model.bse[0]
        lambda_tstat = model.tvalues[0]

        print("CAPM CSR metrics calculated.")
        return {
            "Lambda": lambda_value,
            "Lambda_SE": lambda_se,
            "Lambda_tStat": lambda_tstat
        }

    # Part 2e: Performing Fama Macbeth Procedure
    def calculate_FMP_CAPM(self):
        # List to store lambdas at each time period t
        lambdas_over_time = []

        # Iterating over the time periods
        for t in range(len(self.excess_returns)):
            # Extract the excess return
            excess_returns_t = self.excess_returns.iloc[t, :]

            # Perform the cross-sectional regression for period t (using CAPM Betas)
            X = sm.add_constant(pd.Series(self.capm_betas))
            y = excess_returns_t
            model = sm.OLS(y, X).fit()

            # Extract metric
            lambda_t = model.params[0]
            lambdas_over_time.append(lambda_t)

        # Return lambdas as a Series with time index and metrics
        lambdas_series = pd.Series(lambdas_over_time, index=self.excess_returns.index, name="Lambda_t")
        lambda_hat = lambdas_series.mean()
        lambda_se = lambdas_series.std() / np.sqrt(len(lambdas_series))
        lambda_t_stat = lambda_hat / lambda_se


        print("CAPM FMP results calculated.")
        return {
            "Lambda_hat": lambda_hat,
            "Lambda_SE": lambda_se,
            "Lambda_tStat": lambda_t_stat
        }

    # Part 2f: Performing Time Series Joint Hypothesis Test (GRS)
    def calculate_GRS_CAPM(self):
      # Defining matrix to store residuals
        residuals = pd.DataFrame(index=self.excess_returns.index, columns=self.excess_returns.columns)

        # Calculate residuals for each portfolio and each time period
        for portfolio in self.portfolio_columns:
            # Extract the CAPM alpha and beta for this portfolio
            alpha = self.capm_alphas[portfolio]
            beta = self.capm_betas[portfolio]

            # Calculate the predicted returns for each time period
            predicted_returns = alpha + beta * self.df['RMRF']

            # Compute residuals
            residuals[portfolio] = self.excess_returns[portfolio] - predicted_returns

        # Create a matrix of demeaned residuals
        demeaned_residuals = residuals.subtract(residuals.mean(), axis=1)

        # Define T, N, K
        T = len(self.df)  # num periods
        N = len(self.portfolio_columns)  # num portfolios
        K = 1  # num factors

        # Compute the covariance matrix
        covariance_matrix = (demeaned_residuals.T @ demeaned_residuals) / (T - 1)
        alpha_vector = np.array(list(self.capm_alphas.values()))
        inv_cov_matrix = np.linalg.inv(covariance_matrix)

        # Create a vector of expected returns of the facotrs (only 1 for CAPM)
        mu_vec_factors = np.array([self.df['RMRF'].mean()])

        # Compute the factor covariance matrix
        cov_matrix_factors = np.array([[self.df['RMRF'].var(ddof=1)]])
        inv_cov_matrix_factors = np.linalg.inv(cov_matrix_factors)

        # Following the formula from the notes
        # Calculate the denominator term
        factor_term = mu_vec_factors.T @ inv_cov_matrix_factors @ mu_vec_factors
        second_term_denom = 1 + factor_term

        # Calculate the GRS F-statistic and Critical Value
        first_term = (T - N - K) / N
        second_term_num = alpha_vector.T @ inv_cov_matrix @ alpha_vector
        f_grs = first_term * (second_term_num / second_term_denom)
        f_critical = f.ppf(0.95, N, T - N - K)

        print("CAPM GRS Test Completed.")
        return {
            "F_GRS": f_grs,
            "F_Critical": f_critical
        }

## FAMA FRENCH 3-FACTOR MODEL

In [4]:
# Create a class to be used for all FF3F Calulcations
class EmpiricalTestsFF3F:
    def __init__(self, df):
        # Drop the 'Date' column
        self.df = df.drop(columns=['Date'], errors='ignore')

        # Store portfolio columns, ignoring the factors and RF column
        self.portfolio_columns = self.df.columns[:-4]
        self.RMRF = self.df['RMRF']
        self.SMB = self.df['SMB']
        self.HML = self.df['HML']
        self.RF = self.df['RF']
        self.excess_returns = pd.DataFrame()
        self.expected_returns = []

    # Preliminary functions for getting returns (excess and expected)
    def calculate_excess_returns(self):
        # Calculate the excess returns
        for column in self.portfolio_columns:
            self.excess_returns[column] = self.df[column] - self.RF
        print("Excess returns calculated.")
        return self.excess_returns

    def calculate_expected_returns(self):
        # Calculate the expected returns
        self.expected_returns = self.excess_returns.mean().values
        print("Expected returns calculated.")
        return self.expected_returns

    # Part 3b: Performing Time Series Regression
    def calculate_TSR_FF3F(self):
        # Ensure excess returns are calculated
        if self.excess_returns.empty:
            self.calculate_excess_returns()

        # Create dictionaries to store metrics
        self.ff3f_betas = {}
        self.ff3f_betas_se = {}
        self.ff3f_betas_tstat = {}
        self.ff3f_alphas = {}
        self.ff3f_alphas_se = {}
        self.ff3f_alphas_tstat = {}

        for column in self.portfolio_columns:
            # Performing multi-variate regression using all three factors
            X = sm.add_constant(self.df[['RMRF', 'SMB', 'HML']])
            y = self.excess_returns[column]
            model = sm.OLS(y, X, missing='drop').fit()

            # Store all metrics
            self.ff3f_betas[column] = model.params[['RMRF', 'SMB', 'HML']]
            self.ff3f_betas_se[column] = model.bse[['RMRF', 'SMB', 'HML']]
            self.ff3f_betas_tstat[column] = model.tvalues[['RMRF', 'SMB', 'HML']]
            self.ff3f_alphas[column] = model.params['const']
            self.ff3f_alphas_se[column] = model.bse['const']
            self.ff3f_alphas_tstat[column] = model.tvalues['const']

        # Print results for confirmation
        print("FF3F TSR metrics calculated.")
        return {
            "FF3F Betas": self.ff3f_betas,
            "Beta_SE": self.ff3f_betas_se,
            "Beta_tStat": self.ff3f_betas_tstat,
            "Alphas": self.ff3f_alphas,
            "Alpha Standard Errors": self.ff3f_alphas_se,
            "Alpha t-Statistics": self.ff3f_alphas_tstat
        }

    # Part 3c: Performing Cross Sectional Regression
    def calculate_CSR_FF3F(self):
        # Conversions to align format
        # Extracting matrix of betas of all three facotrs
        ff3f_betas_df = pd.DataFrame(self.ff3f_betas).T
        expected_returns_series = pd.Series(self.expected_returns, index=ff3f_betas_df.index)

        # Performing regression
        X = sm.add_constant(ff3f_betas_df)
        y = expected_returns_series
        model = sm.OLS(y, X).fit()

        # Extract metrics for each factor
        lambdas = model.params[1:]
        lambda_se = model.bse[1:]
        lambda_tstat = model.tvalues[1:]

        print("FF3F CSR metrics calculated.")
        return {
            "Lambdas": lambdas,
            "Lambda_SE": lambda_se,
            "Lambda_tStat": lambda_tstat
        }

    # Part 3d: Performing Fama Macbeth Procedure
    def calculate_FMP_FF3F(self):
        # List to store lambdas at each time period t
        lambdas_over_time = []

        # Iterating over the time periods
        for t in range(len(self.excess_returns)):
            # Extract excess returns
            excess_returns_t = self.excess_returns.iloc[t, :]

            # Perform the regression for using all factor betas
            X = sm.add_constant(pd.DataFrame(self.ff3f_betas).T)
            y = excess_returns_t
            model = sm.OLS(y, X).fit()

            # Extract lambdas for all factors
            lambda_t = model.params[1:]
            lambdas_over_time.append(lambda_t)

        # Convert to df for further metric calculations
        lambdas_df = pd.DataFrame(lambdas_over_time, index=self.excess_returns.index)
        lambda_hat = lambdas_df.mean()
        lambda_se = lambdas_df.std() / np.sqrt(len(lambdas_df))
        lambda_t_stat = lambda_hat / lambda_se


        print("FF3F FMP results calculated.")
        return {
            "Lambdas": lambda_hat,
            "Lambda_SE": lambda_se,
            "Lambda_tStat": lambda_t_stat
        }

    # Part 3e: Performing Time Series Joint Hypothesis Test (GRS)
    def calculate_GRS_FF3F(self):
        # Create a matrix to store residuals
        residuals = pd.DataFrame(index=self.excess_returns.index, columns=self.excess_returns.columns)

        # Calculate residuals for each portfolio and each time period
        for portfolio in self.portfolio_columns:
            # Retrieve the FF3F alpha and betas for this portfolio
            alpha = self.ff3f_alphas[portfolio]
            betas = self.ff3f_betas[portfolio]

            # Calculate the predicted returns for each time period
            predicted_returns = alpha + (betas['RMRF'] * self.df['RMRF'] +
                                         betas['SMB'] * self.df['SMB'] +
                                         betas['HML'] * self.df['HML'])

            # Compute residuals
            residuals[portfolio] = self.excess_returns[portfolio] - predicted_returns

        demeaned_residuals = residuals.subtract(residuals.mean(), axis=1)

        # Define T, N, K
        T = len(self.df)  # num periods
        N = len(self.portfolio_columns)  # num portfolios
        K = 3  # num factors

        # Create the covariance matrix
        covariance_matrix = (demeaned_residuals.T @ demeaned_residuals) / (T - 1)
        alpha_vector = np.array(list(self.ff3f_alphas.values()))
        inv_cov_matrix = np.linalg.inv(covariance_matrix)

        # Create a vector of expected returns of the facotrs
        mu_vec_factors = np.array([self.df['RMRF'].mean(), self.df['SMB'].mean(), self.df['HML'].mean()])

        # Create the factor covariance matrix
        cov_matrix_factors = self.df[['RMRF', 'SMB', 'HML']].cov()
        inv_cov_matrix_factors = np.linalg.inv(cov_matrix_factors)

        # Calculate the denominator term with the factor mean vector and factor covariance matrix
        factor_term = mu_vec_factors.T @ inv_cov_matrix_factors @ mu_vec_factors
        second_term_denom = 1 + factor_term

        # Following the formula, calculate the GRS F-statistic and Critical Value
        first_term = (T - N - K) / N
        second_term_num = alpha_vector.T @ inv_cov_matrix @ alpha_vector
        f_grs = first_term * (second_term_num / second_term_denom)
        f_critical = f.ppf(0.95, N, T - N - K)

        print("FF3F GRS Test completed.")
        return {
            "F_GRS": f_grs,
            "F_Critical": f_critical
        }

## Writing Output To Excel

In [5]:
# works besides tests for CAPM (tried fixing)
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill
import pandas as pd

def write_CAPM_to_excel(wb, capm_csr_results, fmp_results, grs_results, filename="CAPM_Test.xlsx"):
    # Initialize the active sheet in the workbook
    ws = wb.active
    ws.title = "CAPM Test Results"

    # Define styles for headers and cells
    header_font = Font(bold=True, size=14)
    header_fill = PatternFill(start_color="D9D9D9", end_color="D9D9D9", fill_type="solid")
    alignment_center = Alignment(horizontal="center", vertical="center")

    # CAPM Test Header
    ws["A1"] = "CAPM TEST"
    ws["A1"].font = header_font

    # Helper function to style headers
    def style_header(row, headers):
        for col, header in enumerate(headers, start=1):
            cell = ws.cell(row=row, column=col, value=header)
            cell.font = Font(bold=True)
            cell.fill = header_fill
            cell.alignment = alignment_center

    # Function to write DataFrame data to specific location
    def write_df_to_sheet(df, start_row):
        for i, (index, row) in enumerate(df.iterrows(), start=start_row):
            ws[f"A{i}"] = index  # Write the index as "Time Period"
            for j, value in enumerate(row, start=2):
                ws.cell(row=i, column=j, value=value)

    # 1. Cross-Sectional Regression Section
    ws["A3"] = "Cross Sectional Regression: E(Rᵉᵢₜ) = (γ) + βᵢλₜ + αᵢ, i = 1 ... N, ∀t"
    ws["A3"].font = Font(bold=True)

    # Headers for Cross-Sectional Regression Table
    style_header(4, ["Time Period", "Lambda (β)", "Standard Error", "T-Statistic", "Significant?"])
    write_df_to_sheet(capm_csr_results, start_row=5)

    # 2. FMP Regression Section (Place directly after CSR table)
    start_row_fmp = 5 + len(capm_csr_results) + 1
    ws[f"A{start_row_fmp}"] = "FMP Regression: Rᵉᵢₜ = βᵢλₜ + αᵢ, i = 1 ... N, ∀t"
    ws[f"A{start_row_fmp}"].font = Font(bold=True)

    # Headers for FMP Regression Table
    style_header(start_row_fmp + 1, ["Time Period", "Lambda (β)", "Standard Error", "T-Statistic", "Significant?"])
    write_df_to_sheet(fmp_results, start_row=start_row_fmp + 2)

    # 3. GRS F-Test Section (Place directly after FMP table)
    start_row_grs = start_row_fmp + len(fmp_results) + 3
    ws[f"A{start_row_grs}"] = "GRS F-Test: f₍GRS₎ = ((T - N - K) / N) * ((α' Σₑ⁻¹ α) / (1 + μ' Σᵣ⁻¹ μ)) ∼ F(N, T - N - K)"
    ws[f"A{start_row_grs}"].font = Font(bold=True)

    # Headers for GRS F-Test Table
    style_header(start_row_grs + 1, ["Time Period", "f₍GRS₎", "Critical Value", "Reject/Fail to Reject Null?"])
    
    # Write GRS results with reject/fail to reject logic
    for i, period in enumerate(grs_results.index, start=start_row_grs + 2):
        ws[f"A{i}"] = period
        ws[f"B{i}"] = grs_results.loc[period, "F_GRS"]
        ws[f"C{i}"] = grs_results.loc[period, "F_Critical"]
        ws[f"D{i}"] = "Reject" if grs_results.loc[period, "F_GRS"] >= grs_results.loc[period, "F_Critical"] else "Fail to Reject"

    # Save the workbook
    wb.save(filename)
    print(f"Results successfully saved to {filename}")

def write_FF3F_to_excel(wb, ff3f_csr_results, ff3f_fmp_results, ff3f_grs_results):
    # Create a new sheet for FF3F results
    ws = wb.create_sheet(title="FF3F Test Results")

    # Define styles for headers and cells
    header_font = Font(bold=True)
    header_fill = PatternFill(start_color="D9D9D9", end_color="D9D9D9", fill_type="solid")
    alignment_center = Alignment(horizontal="center", vertical="center")

    # FF3F Test Header
    ws.merge_cells("A1:J1")
    ws["A1"] = "FF3F TEST"
    ws["A1"].font = Font(bold=True, size=14)

    # Cross-Sectional Regression Section
    ws.merge_cells("A3:J3")
    ws["A3"] = "Cross Sectional Regression: E(Rᵉᵢₜ) = (γ) + βᵢλₜ + αᵢ, i = 1 ... N, ∀t"
    ws["A3"].font = Font(bold=True)

    # Define factor names based on columns in the results DataFrame
    factors = ff3f_csr_results["Lambdas"].columns

    # Headers for Cross-Sectional Regression Table
    headers = ["Time Period"]
    for factor in factors:
        headers.extend([f"λ_{factor}", f"σ(λ_{factor})", f"t(λ_{factor})"])

    # Write headers to Excel
    for col, header in enumerate(headers, start=1):
        ws.cell(row=4, column=col).value = header
        ws.cell(row=4, column=col).font = header_font
        ws.cell(row=4, column=col).fill = header_fill
        ws.cell(row=4, column=col).alignment = alignment_center

    # Fill Cross-Sectional Regression Table Data
    for i, (period, row) in enumerate(ff3f_csr_results["Lambdas"].iterrows(), start=5):
        ws[f"A{i}"] = period
        col_index = 2
        for factor in factors:
            ws.cell(row=i, column=col_index).value = row[factor]
            ws.cell(row=i, column=col_index + 1).value = ff3f_csr_results["Lambda_SE"].loc[period, factor]
            ws.cell(row=i, column=col_index + 2).value = ff3f_csr_results["Lambda_tStat"].loc[period, factor]
            col_index += 3

    # FMP Regression Section
    start_row_fmp = i + 2  # Place FMP table after CSR table
    ws.merge_cells(f"A{start_row_fmp}:J{start_row_fmp}")
    ws[f"A{start_row_fmp}"] = "FMP Regression: Rᵉᵢₜ = βᵢλₜ + αᵢ, i = 1 ... N, ∀t"
    ws[f"A{start_row_fmp}"].font = Font(bold=True)

    # Write FMP Regression Table Headers
    for col, header in enumerate(headers, start=1):
        ws.cell(row=start_row_fmp + 1, column=col).value = header
        ws.cell(row=start_row_fmp + 1, column=col).font = header_font
        ws.cell(row=start_row_fmp + 1, column=col).fill = header_fill
        ws.cell(row=start_row_fmp + 1, column=col).alignment = alignment_center

    # Fill FMP Regression Table Data
    for i, (period, row) in enumerate(ff3f_fmp_results["Lambdas"].iterrows(), start=start_row_fmp + 2):
        ws[f"A{i}"] = period
        col_index = 2
        for factor in factors:
            ws.cell(row=i, column=col_index).value = row[factor]
            ws.cell(row=i, column=col_index + 1).value = ff3f_fmp_results["Lambda_SE"].loc[period, factor]
            ws.cell(row=i, column=col_index + 2).value = ff3f_fmp_results["Lambda_tStat"].loc[period, factor]
            col_index += 3

    # GRS F-Test Section
    start_row_grs = i + 2  # Place GRS table after FMP table
    ws.merge_cells(f"A{start_row_grs}:D{start_row_grs}")
    ws[f"A{start_row_grs}"] = "GRS F-Test: f₍GRS₎ = ((T - N - K) / N) * ((α' Σₑ⁻¹ α) / (1 + μ' Σᵣ⁻¹ μ)) ∼ F(N, T - N - K)"
    ws[f"A{start_row_grs}"].font = Font(bold=True)

    # Headers for GRS F-Test Table
    grs_headers = ["Time Period", "f₍GRS₎", "Critical Value", "Reject/Fail to Reject Null?"]
    for col, header in enumerate(grs_headers, start=1):
        ws.cell(row=start_row_grs + 1, column=col).value = header
        ws.cell(row=start_row_grs + 1, column=col).font = header_font
        ws.cell(row=start_row_grs + 1, column=col).fill = header_fill
        ws.cell(row=start_row_grs + 1, column=col).alignment = alignment_center

    # Fill GRS F-Test Table Data
    for i, period in enumerate(ff3f_grs_results.index, start=start_row_grs + 2):
        ws[f"A{i}"] = period
        ws[f"B{i}"] = ff3f_grs_results.loc[period, "F_GRS"]
        ws[f"C{i}"] = ff3f_grs_results.loc[period, "F_Critical"]
        ws[f"D{i}"] = "Reject" if ff3f_grs_results.loc[period, "F_GRS"] > ff3f_grs_results.loc[period, "F_Critical"] else "Fail to Reject"

    print("FF3F results successfully written to Excel sheet.")

## Execution

In [6]:
capm_csr_results = []
capm_fmp_results = []
capm_grs_results = []
ff3f_csr_results = []
ff3f_fmp_results = []
ff3f_grs_results = []

# get results for each time period
for i in range(nper):
    print(f'Getting results for time period {i+1}')
    # create df for each time period
    df_1 = df[(df['Date'] >= start_dates[i]) & (df['Date'] <= end_dates[i])]
    
    # CAPM
    capm = EmpiricalTestsCAPM(df_1)
    capm.calculate_excess_returns()
    capm.calculate_expected_returns()
    capm.calculate_TSR_CAPM()
    capm_csr_results.append(capm.calculate_CSR_CAPM())
    capm_fmp_results.append(capm.calculate_FMP_CAPM())
    capm_grs_results.append(capm.calculate_GRS_CAPM())

    # FF3F
    ff3f = EmpiricalTestsFF3F(df_1)
    ff3f.calculate_excess_returns()
    ff3f.calculate_expected_returns()
    ff3f.calculate_TSR_FF3F()
    ff3f_csr_results.append(ff3f.calculate_CSR_FF3F())
    ff3f_fmp_results.append(ff3f.calculate_FMP_FF3F())
    ff3f_grs_results.append(ff3f.calculate_GRS_FF3F())
    print('\n')

# convert results to dfs for writing
capm_csr_results = pd.DataFrame(capm_csr_results, index=periods)
capm_csr_results['Significant?'] = capm_csr_results['Lambda_tStat'].abs() >= 1.96
capm_fmp_results = pd.DataFrame(capm_fmp_results, index=periods)
capm_fmp_results['Significant?'] = capm_fmp_results['Lambda_tStat'].abs() >= 1.96
capm_grs_results = pd.DataFrame(capm_grs_results, index=periods)
data = {outer_key: pd.DataFrame([result[outer_key] for result in ff3f_csr_results]) 
        for outer_key in ff3f_csr_results[0]}
ff3f_csr_results = pd.concat(data, axis=1)
ff3f_csr_results.index = periods
data = {outer_key: pd.DataFrame([result[outer_key] for result in ff3f_fmp_results]) 
        for outer_key in ff3f_fmp_results[0]}
ff3f_fmp_results = pd.concat(data, axis=1)
ff3f_fmp_results.index = periods
ff3f_grs_results = pd.DataFrame(ff3f_grs_results, index=periods)

# Write Results
wb = Workbook()
write_CAPM_to_excel(wb, capm_csr_results, capm_fmp_results, capm_grs_results, filename="Empirical_Testing_Results.xlsx")
write_FF3F_to_excel(wb, ff3f_csr_results, ff3f_fmp_results, ff3f_grs_results)
# Finally, save the workbook
wb.save("Empirical_Testing_Results.xlsx")

Getting results for time period 1
Excess returns calculated.
Expected returns calculated.
CAPM TSR metrics calculated.
CAPM CSR metrics calculated.
CAPM FMP results calculated.
CAPM GRS Test Completed.
Excess returns calculated.
Expected returns calculated.
FF3F TSR metrics calculated.
FF3F CSR metrics calculated.
FF3F FMP results calculated.
FF3F GRS Test completed.


Getting results for time period 2
Excess returns calculated.
Expected returns calculated.
CAPM TSR metrics calculated.
CAPM CSR metrics calculated.
CAPM FMP results calculated.
CAPM GRS Test Completed.
Excess returns calculated.
Expected returns calculated.
FF3F TSR metrics calculated.
FF3F CSR metrics calculated.
FF3F FMP results calculated.
FF3F GRS Test completed.


Results successfully saved to Empirical_Testing_Results.xlsx
FF3F results successfully written to Excel sheet.
